<a href="https://colab.research.google.com/github/Alexsam14/shakespeare-gpt/blob/main/text_generation_using_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# getting data and printing few lines
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()
print(text[:100])

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(vocab_size)

--2025-02-04 11:19:24--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.008s  

2025-02-04 11:19:25 (133 MB/s) - ‘input.txt’ saved [1115394/1115394]

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',

In [2]:
#char to integer mapping and vice versa

#string to integer dictionary
stoi = {}
for i in range(vocab_size):
  stoi[chars[i]]=i

#integer to string dictionary
itos = {}
for i in range(vocab_size):
  itos[i] = chars[i]

def encode(s):
  s_encoded = []
  for i in s:
    s_encoded.append(stoi[i])
  return s_encoded

def decode(i):
  i_decoded = ''
  for j in i:
    i_decoded += itos[j]
  return i_decoded

In [3]:
def get_hyperparameters():
  emb_dim = 64
  num_heads = 8
  num_blocks = 2
  batch_size = 64
  learning_rate = 1e-6
  num_epochs = 10
  context_size = 100
  dropout = 0.3
  return emb_dim, num_heads, num_blocks, batch_size, learning_rate, num_epochs, context_size, dropout

emb_dim, num_heads, num_blocks, batch_size, learning_rate, num_epochs, context_size, dropout = get_hyperparameters()

In [4]:
import torch

#Rotary Position Embedding (RoPE) for relative position encoding for transformer weights
def rope(x, theta_base=10000.0):
  batch_size, seq_len, emb_dim = x.size()
  assert emb_dim % 2 == 0

  pos = torch.arange(0, seq_len, dtype=torch.float32, device=x.device)
  pos = pos.unsqueeze(0).expand(batch_size, seq_len)

  p = torch.arange(1, emb_dim // 2 + 1, dtype=torch.float32, device=x.device)
  theta_p = 1.0 / (theta_base ** (2 * (p-1)/ emb_dim))

  pos = pos.unsqueeze(-1)
  theta = pos * theta_p

  sin_theta = torch.sin(theta)
  cos_theta = torch.cos(theta)

  x1 = x[..., 0::2]
  x2 = x[..., 1::2]

  x_rotated_1 = x1 * cos_theta - x2 * sin_theta
  x_rotated_2 = x1 * sin_theta + x2 * cos_theta

  x_rotated = torch.stack((x_rotated_1, x_rotated_2), dim=-1).reshape(batch_size, seq_len, emb_dim)
  return x_rotated

In [5]:
import torch.nn as nn
import math

class Head(nn.Module):
  def __init__(self, emb_dim, d_h):
    super().__init__()
    self.W_k = nn.Linear(emb_dim, d_h)
    self.W_q = nn.Linear(emb_dim, d_h)
    self.W_v = nn.Linear(emb_dim, d_h)
    self.d_h = d_h

  def forward(self, x, mask=None):
    K = self.W_k(x)
    V = self.W_v(x)
    Q = self.W_q(x)
    Q, K = rope(Q), rope(K)
    attention_scores = torch.matmul(Q, K.transpose(-2,-1)) / math.sqrt(self.d_h)
    if mask is not None:
      masked_scores = attention_scores.masked_fill(mask == 0, float('-inf'))
    probs = torch.softmax(masked_scores, -1)
    attention_weights = torch.matmul(probs, V)
    return attention_weights

In [6]:
class MultiheadAttention(nn.Module):
  def __init__(self, emb_dim, num_heads):

    super().__init__()
    assert emb_dim % num_heads == 0

    self.d_h = emb_dim // num_heads
    self.heads = nn.ModuleList([Head(emb_dim, self.d_h) for _ in range(num_heads)])
    self.W_o = nn.Linear(emb_dim, emb_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask=None):
    head_outputs = [head(x,mask) for head in self.heads]
    multihead_output = torch.cat(head_outputs, dim=-1)
    return self.dropout(self.W_o(multihead_output))

In [7]:
class MLP(nn.Module):
  #Just a feedforward network to increase network width
  def __init__(self, emb_dim):
    super().__init__()
    self.fc1 = nn.Linear(emb_dim, emb_dim * 4)
    self.fc2 = nn.Linear(emb_dim * 4, emb_dim)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [8]:
class DecoderBlock(nn.Module):
  def __init__(self, emb_dim, num_heads):
    super().__init__()
    self.norm1 = nn.LayerNorm(emb_dim)
    self.norm2 = nn.LayerNorm(emb_dim)
    self.mha = MultiheadAttention(emb_dim, num_heads)
    self.mlp = MLP(emb_dim)

  def forward(self, x, mask):
    norm_output = self.norm1(x)
    attention_output = self.mha(norm_output, mask)
    x = x + attention_output
    norm_output = self.norm2(x)
    mlp_output = self.mlp(norm_output)
    x = x + mlp_output
    return x

In [9]:
class DecoderLanguageModel(nn.Module):
  def __init__(self, vocab_size, emb_dim, num_heads, num_blocks):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, emb_dim)
    self.blocks = nn.ModuleList([DecoderBlock(emb_dim, num_heads) for _ in range(num_blocks)])
    self.output = nn.Linear(emb_dim, vocab_size)

  def forward(self, x):
    x = self.embedding(x)

    _, seq_len, _ = x.size()
    mask = torch.tril(torch.ones(seq_len, seq_len)).to(device)

    for block in self.blocks:
      x = block(x, mask)

    return self.output(x)

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)

#data split into training, validation and test sets in ratio 8:1:1 respectively
m = int(0.8*len(data))
n = int(0.1*len(data))

train_set = data[:m]
validation_set = data[m:m+n]
test_set = data[m+n:]

print(f'Number of training tokens: {len(train_set)}')
print(f'Number of validation tokens: {len(validation_set)}')
print(f'Number of testing tokens: {len(test_set)}')
print(f'Total number of tokens: {sum([len(train_set),len(validation_set),len(test_set)])}')

Number of training tokens: 892315
Number of validation tokens: 111539
Number of testing tokens: 111540
Total number of tokens: 1115394


In [11]:
from torch.utils.data import Dataset, DataLoader

class TextSequenceGenerator(Dataset):
  def __init__(self, data, sequence_length):
    self.data = data
    self.sequence_length = sequence_length
    self.n_sequences = len(data) - sequence_length

  def __len__(self):
    return self.n_sequences

  def __getitem__(self, idx):
    #Get input sequence and output sequence (shifted by 1)
    input_sequence = self.data[idx:idx+self.sequence_length]
    target_sequence = self.data[idx+1:idx+self.sequence_length+1]
    return input_sequence, target_sequence

In [12]:
train_dataset = TextSequenceGenerator(train_set, context_size)
validation_dataset = TextSequenceGenerator(validation_set, context_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers=0, drop_last=True)
validation_loader = DataLoader(validation_dataset, batch_size = batch_size, shuffle = True, num_workers=0, drop_last=True)

In [13]:
from tqdm import tqdm

def train_model(model, train_loader, validation_loader, num_epochs, learning_rate,
                device=None, early_stopping_patience=5, scheduler_patience=2
                ):
  if device is None:
    #Use GPU if available otherwise use CPU instead
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model = model.to(device)
  optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=1e-3)
  criterion = nn.CrossEntropyLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
      optimizer, mode='min', patience=scheduler_patience, factor=0.5, verbose=True
      )

  total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print(f"\nTotal trainable parameters: {total_params:,}")
  print(f'Using device: {device}\n')

  #Early stopping setup
  best_val_loss = float('inf')
  patience_counter = 0
  best_model_state = None

  for epoch in range(num_epochs):
    #Training phase
    model.train()
    total_train_loss = 0.0
    train_batches = 0


    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')

    for (xb, yb) in  train_pbar:
      xb, yb = xb.to(device), yb.to(device)

      optimizer.zero_grad()

      #forward pass
      logits = model(xb)
      logits = logits.reshape(-1, logits.size(-1))
      yb = yb.reshape(-1)

      #compute loss and backward pass
      loss = criterion(logits,yb)
      loss.backward()
      optimizer.step()

      #update metrics
      total_train_loss += loss.item()
      train_batches += 1

      #Update progress bar
      train_pbar.set_postfix({'loss' : f'{loss.item():.4f}'})

    avg_train_loss = total_train_loss/train_batches

    #Validation phase
    model.eval()
    total_val_loss = 0.0
    val_batches = 0

    with torch.no_grad():
      val_pbar = tqdm(validation_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')
      for xb,yb in val_pbar:
        xb, yb = xb.to(device), yb.to(device)

        #Forward pass
        val_logits = model(xb)
        val_logits = val_logits.reshape(-1, val_logits.size(-1))
        yb = yb.reshape(-1)

        #Loss computation
        val_loss = criterion(val_logits, yb)

        #Updating metrics
        total_val_loss += val_loss.item()
        val_batches += 1

        #Update progress bar
        val_pbar.set_postfix({'loss': f'{val_loss.item():.4f}'})

    avg_val_loss = total_val_loss/val_batches

    #Print epoch results
    print(f'\nEpoch {epoch+1}/{num_epochs}:')
    print(f'Average Training Loss: {avg_train_loss}')
    print(f'Average Validation Loss: {avg_val_loss}')

    #Learning rate scheduling
    scheduler.step(avg_val_loss)

    #Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        best_model_state = model.state_dict().copy()
        print(f'New best validation loss: {best_val_loss:.4f}')
    else:
        patience_counter += 1
        print(f'Early stopping counter: {patience_counter}/{early_stopping_patience}')

    if patience_counter >= early_stopping_patience:
        print('\nEarly stopping triggered!')
        break

  # Load best model
  if best_model_state is not None:
      model.load_state_dict(best_model_state)
      print(f'\nLoaded best model with validation loss: {best_val_loss:.4f}')

  return model

In [14]:
model = DecoderLanguageModel(vocab_size, emb_dim, num_heads, num_blocks)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = train_model(
    model=model, train_loader=train_loader, validation_loader=validation_loader,
    num_epochs=num_epochs, learning_rate=learning_rate, early_stopping_patience=5
    )

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Total trainable parameters: 108,353
Using device: cuda



Epoch 1/10 [Val]: 100%|██████████| 1741/1741 [00:36<00:00, 47.68it/s, loss=2.9748]



Epoch 1/10:
Average Training Loss: 3.4227553218132796
Average Validation Loss: 2.955871495893808
New best validation loss: 2.9559


Epoch 2/10 [Val]: 100%|██████████| 1741/1741 [00:41<00:00, 42.23it/s, loss=2.6512]


...


Epoch 8/10:
Average Training Loss: 2.2919022650424514
Average Validation Loss: 2.3024110468141927
New best validation loss: 2.3024


Epoch 9/10 [Val]: 100%|██████████| 1741/1741 [00:37<00:00, 46.90it/s, loss=2.2877]



Epoch 9/10:
Average Training Loss: 2.2562254693792063
Average Validation Loss: 2.263692749292942
New best validation loss: 2.2637


Epoch 10/10 [Val]: 100%|██████████| 1741/1741 [00:36<00:00, 47.52it/s, loss=2.2423]


Epoch 10/10:
Average Training Loss: 2.2211287883980204
Average Validation Loss: 2.226141378018173
New best validation loss: 2.2261

Loaded best model with validation loss: 2.2261


In [21]:
def generate_text(model, start_token=None, max_new_tokens=50, temperature=1):

  model.eval()

  token_indices = encode(start_token)

  input_tensor = torch.tensor([token_indices], dtype=torch.long).to(device)

  generated_indices = input_tensor

  for _ in range(max_new_tokens):
    logits = model(generated_indices)
    logits = logits[:,-1,:]/temperature
    probs = torch.softmax(logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    generated_indices = torch.cat((generated_indices, next_token), dim=1)

  return generated_indices

In [26]:
text = 't'

#generating output for various temperatures
generated_output_t1 = generate_text(
    trained_model, start_token = text, max_new_tokens=50, temperature = 1
    )
print(decode(generated_output_t1[0].cpu().tolist()))


generated_output_t0_8 = generate_text(
    trained_model, start_token = text, max_new_tokens=50, temperature = 0.8
   )
print(decode(generated_output_t0_8[0].cpu().tolist()))

t wit,
Breir nofcer abte hif wher thing arond wike 
tery.

RDWARIICEVETD:
Gan him a dell she tounesen I


In [27]:
# Save entire model
torch.save(trained_model, '/content/drive/MyDrive/Models/shakespeare-gpt-model.pth')

# Save just the state dict (recommended)
torch.save(trained_model.state_dict(), 'shakespeare-gpt-model_state.pth')